In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# 데이터를 불러온다
train_df = pd.read_csv("/kaggle/input/titanic/train.csv")
test_df = pd.read_csv("/kaggle/input/titanic/test.csv")
submission = pd.read_csv("/kaggle/input/titanic/gender_submission.csv")

In [ ]:
# 난수 생성
import random
np.random.seed(1234)
random.seed(1234)

In [ ]:
# 데이터프레임의 차원을 확인한다
print(train_df.shape)
print(test_df.shape)

In [ ]:
#가독성을 위해 열, 행 수를 50으로 제한
pd.set_option("display.max_columns", 50)
pd.set_option("display.max_rows", 50)

In [ ]:
train_df.head(10)

In [ ]:
test_df.head(10)

In [ ]:
# 자료형 파악
train_df.dtypes

In [ ]:
# 통계를 요약해서 확인함
train_df.describe()

In [ ]:
test_df.describe()

In [ ]:
# 데이터 불균형 여부 확인
train_df["Sex"].value_counts()

In [ ]:
train_df["Embarked"].value_counts()

In [ ]:
train_df["Cabin"].value_counts()

In [ ]:
# 데이터의 결측치 확인
train_df.isnull().sum()

In [ ]:
test_df.isnull().sum()

In [ ]:
# EDA 단계에서 필요한 라이브러리 임트트
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:

plt.style.use("ggplot")

In [ ]:
# 데이터에서 필요한 열을 추출한다
train_df[["Embarked", "Survived", "PassengerId"]]

In [ ]:
# 필요한 행을 추출하고 결측치가있는 행을 거거
train_df[["Embarked", "Survived", "PassengerId"]].dropna()

In [ ]:
# 탑승항과 생존 여부별 승객 분포를 카운트한다
train_df[["Embarked", "Survived", "PassengerId"]].dropna().groupby(["Embarked", "Survived"]).count()

In [ ]:
# 탑승항과 생존 여부를 표로 만듦.
embarked_df = train_df[["Embarked", "Survived", "PassengerId"]].dropna().groupby(["Embarked", "Survived"]).count().unstack()
embarked_df

In [ ]:
# 막대그래프 생성
embarked_df.plot.bar(stacked=True)

In [ ]:
# 탑승항 별 생존률을 계산해 테이블에 추가
embarked_df["survived_rate"] = embarked_df.iloc[:,0] / (embarked_df.iloc[:,0] + embarked_df.iloc[:,1])
embarked_df

In [ ]:
# 성별로 사망 여부 집계 후 막대그래프 그림
sex_df = train_df[["Sex", "Survived", "PassengerId"]].dropna().groupby(["Sex", "Survived"]).count().unstack()
sex_df.plot.bar(stacked=True)

In [ ]:
# 객실 등급 별 사망 여부 집계 후 막대 그래프
ticket_df = train_df[["Pclass", "Survived", "PassengerId"]].dropna().groupby(["Pclass", "Survived"]).count().unstack()
ticket_df.plot.bar(stacked=True)

In [ ]:
# 연령 별 사망 여부 집계 후 막대그래프
age_df = train_df[["Age", "Survived", "PassengerId"]].dropna().groupby(["Age", "Survived"]).count().unstack()
age_df.plot.bar(stacked=True)

In [ ]:
# 생존자와 사망자의 나이 분포 히스토그램
plt.hist(x=[train_df.Age[train_df.Survived == 0], train_df.Age[train_df.Survived==1]], bins=8, histtype='barstacked', label=["Death", "Survived"])
plt.legend()

In [ ]:
# 성별, 탑승항 을 0/1 더미 변수로 인코딩
train_df_corr = pd.get_dummies(train_df, columns=["Sex"], drop_first=True)
train_df_corr = pd.get_dummies(train_df_corr, columns=["Embarked"])
train_df_corr.head()

In [ ]:
# 모델 입력용 변수 사이 상관계수를 확인
train_corr = train_df_corr.select_dtypes(include=["number", "bool"]).corr()
train_corr

In [ ]:
# 9x9 사이즈에 숫자 주석이 포함된 상관관계 히트맵을 그린다
plt.figure(figsize=(9, 9))
sns.heatmap(train_corr, vmax=1, vmin=-1, center=0, annot=True)

In [ ]:
# 학습, 테스트 세트를 하나로 묶는다
all_df = pd.concat([train_df, test_df], sort=False).reset_index(drop=True)
all_df

In [ ]:
# 결측치를 열 단위로 확인한다 
all_df.isnull().sum()

In [ ]:
# 결측치를 채우기 위해 객실 등급별 평균 요금을 계산해 채운다
Fare_mean = all_df[["Pclass", "Fare"]].groupby("Pclass").mean().reset_index()

Fare_mean.columns = ["Pclass", "Fare_mean"]

Fare_mean

In [ ]:
# 등급 별 평균 요금을 이용해 결측치를 대체한다
all_df = pd.merge(all_df, Fare_mean, on="Pclass", how="left")

all_df.loc[(all_df["Fare"].isnull()), "Fare"] = all_df["Fare_mean"]

all_df = all_df.drop("Fare_mean", axis=1)

In [ ]:
# 승객 이름 행에서 표본을 추출한다
all_df["Name"].head()

In [ ]:
# 이름 컬럼을 분리해 새 변수를 만든다. 최대 세 개의 이름을 얻는다 . 성 이름 등
name_df = all_df["Name"].str.split("[,.]", n = 2, expand=True)

# 분리된 세 컬럼으로 새로운 열을 만든다
name_df.columns = ["family_name", "honorific", "name"]

name_df

In [ ]:
# 위에서 만든 컬럼의 공백을 제거한다
name_df["family_name"] = name_df["family_name"].str.strip()
name_df["honorific"] = name_df["honorific"].str.strip()
name_df["name"] = name_df["name"].str.strip()

In [ ]:
# 호칭 별로 빈도 분포를 확인한다
name_df["honorific"].value_counts()

In [ ]:
# 위에서 만든 컬럼을 원본 데이터프레임에 결합한다
all_df = pd.concat([all_df, name_df], axis=1)

all_df

In [ ]:
# 호칭 별 나이 분포를 시각화한다
plt.figure(figsize=(18, 5))
sns.boxplot(x="honorific", y="Age", data=all_df)

In [ ]:
# name.df를 분할하 후 각각 결합하여 두 데이터 셋에 성 호칭 이름 컬럼을 추가한다
train_df = pd.concat([train_df, name_df[0:len(train_df)].reset_index(drop=True)], axis=1)
test_df = pd.concat([test_df, name_df[len(train_df):]])

In [ ]:
# 호칭 별 사망 여부를 집계 하여 막대그래프로 표현
honorific_df = train_df[["honorific", "Survived", "PassengerId"]].dropna().groupby(["honorific", "Survived"]).count().unstack()
honorific_df.plot.bar(stacked=True)

In [ ]:
# 호칭 별 평균 나이를 계산하여 데이터 프레임을 생성
honorific_age_mean = all_df[["honorific", "Age"]].groupby("honorific").mean().reset_index()

# 평균 나이 칼럼을 변경한다
honorific_age_mean.columns = ["honorific", "honorific_Age"]

# 호칭별 평균 나이를 원본 데이터 프레임에 붙인다
all_df = pd.merge(all_df, honorific_age_mean, on="honorific", how="left")

# age 결측치를 호칭별 평균 나이로 채운다
all_df.loc[all_df["Age"].isnull(), "Age"] = all_df["honorific_Age"]

# 임시 컬럼을 삭제해 정리
all_df = all_df.drop(["honorific_Age"], axis=1)

In [ ]:
# 가족 규모를 새 열로 추가하여 가족 규모별 승객 수를 집계한다
all_df["family_num"] = all_df["Parch"] + all_df["SibSp"]
all_df["family_num"].value_counts()

In [ ]:
# 동승인이 0인 경우를 찾아 alone 값을 1로 설정한다 그 후 alone 값이 1이 아닌 승객을 0으로 채움
all_df.loc[all_df["family_num"]==0, "alone"] = 1
all_df["alone"] = all_df["alone"].fillna(0)

In [ ]:
# 필요 ㅇ벗는 열을 제거한다
all_df = all_df.drop(["PassengerId", "Name", "family_name", "name", "Ticket", "Cabin"], axis = 1)

all_df.head()

In [ ]:
# 데이터 타입이 문자열인 열 이름만 골라서 저장한다
categories = all_df.columns[all_df.dtypes=="object"]
print(categories)

In [ ]:
# 호칭 빈도가 높지 않은 호칭을 other로 통합, 호칭 별 승객 수를 내림차순으로 집계
all_df.loc[~((all_df["honorific"]=="Mr") | (all_df["honorific"]=="Miss") | (all_df["honorific"]=="Mr") | (all_df["honorific"]=="Master")), "honorific"] = "other"

all_df.honorific.value_counts()

In [ ]:

from sklearn.preprocessing import LabelEncoder

In [ ]:
# 탑승 항구 정보가 빈 승객에게 mising 값을 부여해 결측치 처리
all_df["Embarked"].fillna("missing", inplace=True)
all_df.head()

In [ ]:
# 성별 문자열을 0,1 숫자로 바꿔 수치형으로 인식할수 있게 한다
le = LabelEncoder()
le = le.fit(all_df["Sex"])
all_df["Sex"] = le.transform(all_df["Sex"])

all_df.head()

In [ ]:
# 모든 문자열 컬럼을 labelEncoder로 정수화하여 object 타입이 남지 않게 함
for cat in categories:
    le = LabelEncoder()
    print(cat)
    if all_df[cat].dtypes == "object":
        le = le.fit(all_df[cat])
        all_df[cat] = le.transform(all_df[cat])

all_df.head()

In [ ]:
# 생존 여부로 학습 테스트 세트를 구분해서 train은 모델 학습용 입력과 정답이고 test는 캐글 제출용 입력 데이터를 준비한다
train_X = all_df[~all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)

# Train data (target)
train_Y = train_df["Survived"]

# Test data (features only, no labels)
test_X = all_df[all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [ ]:
# 학습 데이터 중 20%를 검증 세트로 사용
X_train, X_valid, y_train, y_valid = train_test_split(train_X, train_Y, test_size=0.2)

In [ ]:
# LightGBM 전용 데이터 셋을 생성하고 주요 범주형 특성들을 지정해 향후 모델 학습 시 자동으로 카테고리 처리되도록 설정
categories = ["Embarked", "Pclass", "Sex", "honorific", "alone"]
lgb_train = lgb.Dataset(X_train, y_train, categorical_feature=categories)
lgb_eval = lgb.Dataset(X_valid, y_valid, categorical_feature=categories, reference=lgb_train)

In [ ]:
# 주요 파라미터를 변수에 저장
lgbm_params = {
    "objective":"binary",
    "random_seed":1234
}

In [ ]:
# 
model_lgb = lgb.train(lgbm_params,
                      lgb_train,
                      num_boost_round=100,
                      valid_sets=[lgb_eval],
                      callbacks=[
                          lgb.early_stopping(stopping_rounds=20),
                          lgb.log_evaluation(period=10)
                      ]
)

In [ ]:
# Check importance of variable
model_lgb.feature_importance()

In [ ]:
# Importance visualization 
importance = pd.DataFrame(model_lgb.feature_importance(), index=X_train.columns, columns=["importance"]).sort_values(by="importance", ascending=True)
importance.plot.barh()

In [ ]:
# Apply the model to eval data
y_pred = model_lgb.predict(X_valid, num_iteration=model_lgb.best_iteration)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
# Calculating initial accuracy
accuracy_score(y_valid, np.round(y_pred))

In [ ]:
# Hyperparameter optimization
lgbm_params = {
    "objective": "binary",
    "learning_rate": 0.05,
    "random_seed": 1234
}

In [ ]:
# Retrain
lgb_train = lgb.Dataset(X_train, y_train, categorical_feature=categories)
lgb_eval = lgb.Dataset(X_valid, y_valid, categorical_feature=categories, reference=lgb_train)

In [ ]:
# Retrain Machine Learning Model
model_lgb = lgb.train(lgbm_params,
                      lgb_train,
                      num_boost_round=100,
                      valid_sets=[lgb_eval],
                      callbacks=[
                          lgb.early_stopping(stopping_rounds=20),
                          lgb.log_evaluation(period=10)
                      ]
)

In [ ]:
# Recheck importance of variable
model_lgb.feature_importance()

In [ ]:
# Recheck importance visualization 
importance = pd.DataFrame(model_lgb.feature_importance(), index=X_train.columns, columns=["importance"]).sort_values(by="importance", ascending=True)
importance.plot.barh()

In [ ]:
# Apply the model to eval data
y_pred = model_lgb.predict(X_valid, num_iteration=model_lgb.best_iteration)

In [ ]:
# Calculating accuracy
accuracy_score(y_valid, np.round(y_pred))

In [ ]:
# Cross validation
folds = 3
kf = KFold(n_splits=folds)

In [ ]:
# Save accuracy
accuracy_list = []

# Cross validation learning
models = []
for train_index, val_index in kf.split(train_X):
    X_train = train_X.iloc[train_index]
    X_valid = train_X.iloc[val_index]
    y_train = train_Y.iloc[train_index]
    y_valid = train_Y.iloc[val_index]

    lgb_train = lgb.Dataset(X_train, y_train, categorical_feature=categories)
    lgb_eval = lgb.Dataset(X_valid, y_valid, categorical_feature=categories, reference=lgb_train)

    model_lgb = lgb.train(
        lgbm_params,
        lgb_train,
        num_boost_round=100,
        valid_sets=[lgb_eval],
        callbacks=[
            lgb.early_stopping(stopping_rounds=20),
            lgb.log_evaluation(period=10)
        ]
    )

    y_pred = model_lgb.predict(X_valid, num_iteration=model_lgb.best_iteration)
    acc = accuracy_score(y_valid, np.round(y_pred))
    print("Fold accuracy:", acc)
    accuracy_list.append(acc)

    models.append(model_lgb)

In [ ]:
# Average accuracy output
mean_accuracy = np.mean(accuracy_list)
print("Average Cross-Validation Accuracy:", mean_accuracy)

In [ ]:
# Save prediction result
preds = []

for model in models:
    pred = model.predict(test_X)
    preds.append(pred)

In [ ]:
# Calculate mean
preds_array = np.array(preds)
preds_mean = np.mean(preds_array, axis=0)

In [ ]:
# Convert to 0 or 1
preds_int = (preds_mean > 0.5).astype(int)

In [ ]:
# Create submission file
submission["Survived"] = preds_int
submission

In [ ]:
# Save to csv file
submission.to_csv("titanic_submit01.csv", index=False)